In [1]:
DATA_NAME = 'spaceNet-full' 
TRANSFORM = 'wavelet-diagonal'
CHANNEL = 'green'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_green_diagonal_wavelet_full_spaceNet.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9
2,10596.69,-0.16,-0.04,-0.02,0.00,-0.00,0.00,-0.00
3,-0.16,1561.08,0.02,-0.01,0.00,-0.00,-0.00,0.00
4,-0.04,0.02,680.74,-0.00,0.01,0.00,-0.00,-0.00
5,-0.02,-0.01,-0.00,188.51,-0.00,-0.00,0.00,0.00
6,0.00,0.00,0.01,-0.00,44.47,0.00,0.00,0.00
7,-0.00,-0.00,0.00,-0.00,0.00,10.03,-0.00,-0.00
8,0.00,-0.00,-0.00,0.00,0.00,-0.00,2.18,0.00
9,-0.00,0.00,-0.00,0.00,0.00,-0.00,0.00,0.39


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9
2,1.00000,-0.00004,-0.00001,-0.00002,0.00000,-0.00001,0.00001,-0.00003
3,-0.00004,1.00000,0.00001,-0.00002,0.00001,-0.00001,-0.00002,0.00000
4,-0.00001,0.00001,1.00000,-0.00001,0.00005,0.00005,-0.00001,-0.00005
5,-0.00002,-0.00002,-0.00001,1.00000,-0.00001,-0.00001,0.00002,0.00000
6,0.00000,0.00001,0.00005,-0.00001,1.00000,0.00002,0.00000,0.00006
7,-0.00001,-0.00001,0.00005,-0.00001,0.00002,1.00000,-0.00001,-0.00001
8,0.00001,-0.00002,-0.00001,0.00002,0.00000,-0.00001,1.00000,0.00003
9,-0.00003,0.00000,-0.00005,0.00000,0.00006,-0.00001,0.00003,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.23269481331869554

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[1.40439511e+07 3.00325618e+05 5.50113849e+04 4.06143749e+03
 2.12406877e+02 9.64551014e+00 3.45676110e-01 2.45773237e-32]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7
0,0.999723,-0.021498,-0.009182,-0.002532,-0.000597,-0.000135,-0.000029,-0.000005
1,0.020544,0.995650,-0.088644,-0.019476,-0.004519,-0.001020,-0.000222,-0.000039
2,0.010837,0.087016,0.994142,-0.061700,-0.013266,-0.002974,-0.000648,-0.000116
3,0.003546,0.024337,0.058755,0.995878,-0.063118,-0.013293,-0.002875,-0.000513
4,0.001036,0.007037,0.016172,0.060804,0.994691,-0.079444,-0.016140,-0.002861
5,0.000309,0.002093,0.004793,0.017366,0.076044,0.990292,-0.113381,-0.018828
6,0.000098,0.000669,0.001533,0.005536,0.023502,0.106648,0.977830,-0.178574
7,0.000036,0.000245,0.000562,0.002025,0.008580,0.038074,0.175282,0.983742


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.0002766655215906688,
 0.004349792956581111,
 0.005857891901232315,
 0.004121556830701367,
 0.005308755527392739,
 0.009707861913052063,
 0.022169937118887706,
 0.01625797885317637]